In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
TAG_SIZE = 2
VOCAB_SIZE = 20000

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def create_data_generatorr(dataset):
  def data_generator():
    for i in range(len(dataset)):
      left = dataset.loc[i]['x'].split(' ')
      right = None
      if dataset.loc[i]['y'] == 0:
        right = [0 for i in range(len(left))]
      else:
        right = [1 for i in range(len(left))]
      yield left, right
  
  return data_generator

df = pd.read_csv('../scraper/datasets/latest.csv')

le = LabelEncoder()
df['truth_value'] = le.fit_transform(df['truth_value'])

xtrain, xtest, ytrain, ytest = train_test_split(df['claim'], df['truth_value'], test_size=0.2, random_state=42)

train = pd.concat({'x': xtrain, 'y': ytrain}, axis=1)
train.reset_index(drop=True, inplace=True)

splittrain = train['x'].apply(lambda x: x.split(' '))
xtrain_tokens = tf.ragged.constant(train['x'].apply(lambda x: x.split(' ')))
xtrain_tokens = tf.map_fn(tf.strings.lower, xtrain_tokens)

trainlookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
trainlookup_layer.adapt(xtrain_tokens)


data_signature= (
        tf.TensorSpec(shape=(None,), dtype=tf.string),
        tf.TensorSpec(shape=(None, ), dtype=tf.int32)
)

train_data = tf.data.Dataset.from_generator(
    create_data_generatorr(train),
    output_signature=data_signature
)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def create_data_generatorr(dataset):
  def data_generator():
    for i in range(len(dataset)):
      left = dataset.loc[i]['x'].split(' ')
      right = None
      if dataset.loc[i]['y'] == 0:
        right = [0 for i in range(len(left))]
      else:
        right = [1 for i in range(len(left))]
      yield left, right
  
  return data_generator

df = pd.read_csv('../scraper/datasets/latest.csv')

le = LabelEncoder()
df['truth_value'] = le.fit_transform(df['truth_value'])

xtrain, xtest, ytrain, ytest = train_test_split(df['claim'], df['truth_value'], test_size=0.2, random_state=42)

train = pd.concat({'x': xtrain, 'y': ytrain}, axis=1)
train.reset_index(drop=True, inplace=True)
test = pd.concat({'x': xtest, 'y': ytest}, axis=1)
test.reset_index(drop=True, inplace=True)

splittrain = train['x'].apply(lambda x: x.split(' '))
splittest = test['x'].apply(lambda x: x.split(' '))
xtrain_tokens = tf.ragged.constant(train['x'].apply(lambda x: x.split(' ')))
xtrain_tokens = tf.map_fn(tf.strings.lower, xtrain_tokens)
xtest_tokens = tf.ragged.constant(test['x'].apply(lambda x: x.split(' ')))
xtest_tokens = tf.map_fn(tf.strings.lower, xtest_tokens)

trainlookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
trainlookup_layer.adapt(xtrain_tokens)
testlookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
testlookup_layer.adapt(xtest_tokens)


data_signature= (
        tf.TensorSpec(shape=(None,), dtype=tf.string),
        tf.TensorSpec(shape=(None, ), dtype=tf.int32)
)

train_data = tf.data.Dataset.from_generator(
    create_data_generatorr(train),
    output_signature=data_signature
)
test_data = tf.data.Dataset.from_generator(
    create_data_generatorr(test),
    output_signature=data_signature
)

In [5]:

xtrain, xtest, ytrain, ytest = train_test_split(df['simple_sentence'], df['truth_value'], test_size=0.2, random_state=42)

train = pd.concat({'x': xtrain, 'y': ytrain}, axis=1)
train.reset_index(drop=True, inplace=True)
test = pd.concat({'x': xtest, 'y': ytest}, axis=1)
test.reset_index(drop=True, inplace=True)

splittrain = train['x'].apply(lambda x: x.split(' '))
splittest = test['x'].apply(lambda x: x.split(' '))
xtrain_tokens = tf.ragged.constant(train['x'].apply(lambda x: x.split(' ')))
xtrain_tokens = tf.map_fn(tf.strings.lower, xtrain_tokens)
xtest_tokens = tf.ragged.constant(test['x'].apply(lambda x: x.split(' ')))
xtest_tokens = tf.map_fn(tf.strings.lower, xtest_tokens)

trainlookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
trainlookup_layer.adapt(xtrain_tokens)
testlookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
testlookup_layer.adapt(xtest_tokens)


data_signature= (
        tf.TensorSpec(shape=(None,), dtype=tf.string),
        tf.TensorSpec(shape=(None, ), dtype=tf.int32)
)

train_data = tf.data.Dataset.from_generator(
    create_data_generatorr(train),
    output_signature=data_signature
)
test_data = tf.data.Dataset.from_generator(
    create_data_generatorr(test),
    output_signature=data_signature
)

In [6]:
xxtest = xtest.reset_index(drop=True)
yytest = ytest.reset_index(drop=True)
xxtest.loc[0], yytest.loc[0]

('A Culture in the Cross Hairs.Russia’s invasion has systematically destroyed Ukrainian cultural sites.A Times investigation has identified 339 that sustained substantial damage this year..',
 1)

In [7]:
xxxtest = pd.DataFrame({'x': xxtest}, columns=['x'])
xxxtest

,x
0,A Culture in the Cross Hairs.Russia’s invasion...
1,"Amid Mounting Frustration With Germany, Ukrain..."
2,Biden does not want to introduce a no-fly zone...
3,"Says of disclosing his $20,000 speech about Sy..."
4,I’m the Prime Minister of Estonia.Putin Can’t ...
...,...
1082,Davos Worries About a ‘Polycrisis’.So many glo...
1083,Russia is taking two steps to de-escalate the ...
1084,Justice Dept.Announces Task Force to Go After ...
1085,BMW factories in Europe suspend work due to a ...


In [14]:
i = 0
for a, b in train_data:
  print(' '.join([aa.decode('UTF-8') for aa in a.numpy()]), b)
  i += 1
  if i>10:
    break

What Ukraine Teaches Us About Power.Acting tough doesn’t make you strong, even militarily.. tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(13,), dtype=int32)
After the stunning success of a Ukrainian counteroffensive. Momentum in the war has shifted to Kyiv. Russia is showing no signs of giving up. . . tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(26,), dtype=int32)
Additional troops were sent to the border of the Kursk region after shelling from Ukraine - the governor . tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(19,), dtype=int32)
"The United States has the highest incarceration rate in the world.". tf.Tensor([1 1 1 1 1 1 1 1 1 1 1], shape=(11,), dtype=int32)
The situation in Ukraine after the attacks on infrastructure facilities is critical - Zelenskyi's office . tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(16,), dtype=int32)
Biden intends to tell Xi Jinping that the US will take action if it turns out that China supports Russia's action

In [8]:
def dataset_preprocess_train(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens_test(tokens)
    return preprocessed_tokens, tag_ids

def preprecess_tokens_train(tokens):
    tokens = tf.strings.lower(tokens)
    return trainlookup_layer(tokens)

def dataset_preprocess_test(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens_test(tokens)
    return preprocessed_tokens, tag_ids

def preprecess_tokens_test(tokens):
    tokens = tf.strings.lower(tokens)
    return testlookup_layer(tokens)

BATCH_SIZE = 2048

train_dataset = (
    train_data.map(dataset_preprocess_train)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)
test_dataset = (
    test_data.map(dataset_preprocess_test)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)


In [9]:
# Build the model
def build_embedding_bilstm_crf_model(
    vocab_size: int, embed_dims: int, lstm_unit: int, tag_size: int
) -> tf.keras.Model:
    x = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="x")
    y = tf.keras.layers.Embedding(vocab_size, embed_dims, mask_zero=True)(x)
    y = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(lstm_unit, return_sequences=True)
    )(y)
    decode_sequence, potentials, sequence_length, kernel = tfa.layers.CRF(tag_size)(y)
    return tf.keras.Model(
        inputs=x, outputs=[decode_sequence, potentials, sequence_length, kernel]
    )

model = None
optimizer = None

In [10]:
@tf.function
def crf_loss_func(potentials, sequence_length, kernel, y):
    crf_likelihood, _ = tfa.text.crf_log_likelihood(
        potentials, y, sequence_length, kernel
    )
    # likelihood to loss
    flat_crf_loss = -1 * crf_likelihood
    crf_loss = tf.reduce_mean(flat_crf_loss)

    return crf_loss

train_loss = tf.keras.metrics.Mean(name="train_loss")

@tf.function(experimental_relax_shapes=True)
def train_step(x, y):
    global cur
    global model
    global optimizer
    with tf.GradientTape() as tape:
        decoded_sequence, potentials, sequence_length, kernel = model(x)
        crf_loss = crf_loss_func(potentials, sequence_length, kernel, y)
        loss = crf_loss + tf.reduce_sum(model.losses)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)


In [19]:
import tensorflow_addons as tfa
model = build_embedding_bilstm_crf_model(VOCAB_SIZE, 32, 64, 2)
optimizer = tf.keras.optimizers.Adam(0.02)


for epoch in range(15):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()

  for x, y in train_dataset:
      print(x.shape)
      train_step(x, y)
      # train_step2(x, y)
  print(f"Epoch {epoch + 1}, " f"Loss: {train_loss.result()}")


(2048, 489)
(2048, 261)
(249, 134)
Epoch 1, Loss: 9.534712791442871
(2048, 489)
(2048, 261)
(249, 134)
Epoch 2, Loss: 6.2550482749938965
(2048, 489)
(2048, 261)
(249, 134)
Epoch 3, Loss: 3.819411039352417
(2048, 489)


In [57]:
o = []
oy = []

def processs(x, pred):
  res = []
  for i in range(len(x)):
    b = pred[i]
    tot = (sum(b) / len(b)) * 100
    label = 1
    if tot <= 50:
      label = 0
    res.append(label)
  return res



In [ ]:
from tqdm import tqdm

EPOCHS = 25

# models = [None for i in range(EPOCHS+1)]
# optimizers = [None for i in range(EPOCHS+1)]
cur = 20
for e in range(cur, EPOCHS):
    model = build_embedding_bilstm_crf_model(VOCAB_SIZE, 32, 64, 2)
    optimizer = tf.keras.optimizers.Adam(0.02)

    for epoch in range(e):
      # Reset the metrics at the start of the next epoch
      train_loss.reset_states()

      for x, y in train_dataset:
          train_step(x, y)

      print(f"Epoch {epoch + 1}, " f"Loss: {train_loss.result()}")
    
      acc = 0
      counter = 0
      for i, row in tqdm(xxxtest.iterrows(), total=xxxtest.shape[0]):
        tx = xxtest.loc[i]
        ty = yytest.loc[i]
        preprocessed_inputs = preprecess_tokens_train(tx.split(' '))
        # expend the batch dim
        inputs = tf.reshape(preprocessed_inputs, shape=[1, -1])

        outputs, *_ = model.predict(inputs, verbose=0)
        tot = (sum(outputs[0]) / len(outputs[0])) * 100
        label = 1
        if tot <= 50:
          label = 0
        # print(ty, label)
        if ty == label:
          acc += 1
        # if counter > 100:
        #   break
        counter += 1
      acc /= len(xxtest)
      print('\n', acc)
      cur += 1

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()

    for x, y in train_dataset:
        train_step(x, y)

    print(f"Epoch {epoch + 1}, " f"Loss: {train_loss.result()}")


In [12]:
from tensorflow_addons.layers import CRF
m1 = tf.keras.models.load_model('./m1.h5', custom_objects={"CRF": CRF})
m2 = tf.keras.models.load_model('./m2.h5', custom_objects={"CRF": CRF})


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [59]:
from sklearn.metrics import classification_report, confusion_matrix
o = []
oy = []
m1preds = []
m2preds = []
for x, y in train_dataset:
  print(x.shape, y.shape)
  # print(y, y.shape)
  outputs1, *_ = m1.predict(x)
  outputs2, *_ = m2.predict(x)
  a = processs(x.numpy(), outputs1)
  b = processs(x.numpy(), outputs2)
  m1preds += a
  m2preds += b
  print(len(x.numpy()), len(a), len(b))
  o += zip(tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=500), a, b)
  oy += [p[0] for p in y.numpy()]

print(classification_report(oy, m1preds))
print(confusion_matrix(oy, m1preds))
print(classification_report(oy, m2preds))
print(confusion_matrix(oy, m2preds))

(2048, 489) (2048, 489)
64/64 [==============================] - 8s 127ms/step
2048 2048 2048
(2048, 261) (2048, 261)
64/64 [==============================] - 6s 94ms/step
2048 2048 2048
(249, 134) (249, 134)
8/8 [==============================] - 1s 70ms/step
249 249 249
              precision    recall  f1-score   support

           0       0.10      1.00      0.19       440
           1       0.98      0.01      0.03      3905

    accuracy                           0.11      4345
   macro avg       0.54      0.51      0.11      4345
weighted avg       0.89      0.11      0.04      4345

[[ 439    1]
 [3855   50]]
              precision    recall  f1-score   support

           0       0.10      1.00      0.19       440
           1       0.98      0.01      0.02      3905

    accuracy                           0.11      4345
   macro avg       0.54      0.51      0.11      4345
weighted avg       0.89      0.11      0.04      4345

[[ 439    1]
 [3856   49]]


In [51]:
o = [op[0].tolist()+[op[1],op[2]] for op in o]
print([len(op) for op in o])

[502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502,

In [52]:
print(o[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 2655, 7, 2, 1334, 5008, 58, 20, 3473, 805, 30, 1780, 3695, 147, 490, 20, 4915, 6260, 14, 1485, 3529, 1778, 56, 1420, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [53]:
from sklearn import svm

clf = svm.SVC()
clf.fit(o, oy)

SVC()

In [54]:
from sklearn.metrics import classification_report, confusion_matrix
preds = clf.predict(o)
print(classification_report(oy, preds))
print(confusion_matrix(oy, preds))

              precision    recall  f1-score   support

           0       1.00      0.02      0.05       126
           1       0.89      1.00      0.94       961

    accuracy                           0.89      1087
   macro avg       0.94      0.51      0.49      1087
weighted avg       0.90      0.89      0.84      1087

[[  3 123]
 [  0 961]]
